# Simple Linear Program

Let's solve the following linear program:

\begin{array}{ll}
  \min       & 2 x_1 + 3 x_2\\
  \mathrm{s.t.} & 3 x_1 + 4 x_2 \geq 1\\
             & x_1, x_2 \geq 0
  \end{array}

Before we start, what are the model "parameters"?

### Pyomo model, one chunk at a time

#### Note:  If you try to re-run certain cells you might get an error.  If so, start evaluating cells from the beginning (here).

Import the `pyomo.environ` model as `pe`. 

In [118]:
import pyomo.environ as pe

Create a new (and empty) pyomo model.  The "concrete" means we will hard-code parameter values.

In [119]:
model = pe.ConcreteModel()
print(type(model))

<class 'pyomo.core.base.PyomoModel.ConcreteModel'>


First let's define the decision variables.  We'll give the decisions specific indexes and specify that they be nonnegative.

In [120]:
# We have to tell pyomo how many dv's we have using indexes.  This is a step you will have to
# do yourself most times!  Convention is to use 1-based indexing (ugh) even though python is 0-indexed.
x_indexes = [1, 2]

In [121]:
# Create a variable using the indexes and specifying the type of variable, here nonnegative real numbers, x >= 0.
# The model.x says to attach the variable to the model and call it "x".
model.x = pe.Var(x_indexes, domain=pe.NonNegativeReals)
print(type(model.x))

<class 'pyomo.core.base.var.IndexedVar'>


In [122]:
# Define the objective function value as z = 2x1 + 3x2 and attach it to the model as an attribute obj.
# By default, the objective is set to be a "min" problem.
model.obj = pe.Objective(expr = 2*model.x[1] + 3*model.x[2])
print(type(model.obj))

<class 'pyomo.core.base.objective.SimpleObjective'>


In [123]:
# Create a constraint "3x1 + 4x2 >= 1" and store attach it to the model as Constraint1.
model.constraint1 = pe.Constraint(expr = 3*model.x[1] + 4*model.x[2] >= 1)
print(type(model.constraint1))

<class 'pyomo.core.base.constraint.SimpleConstraint'>


In [124]:
# Instantiate the model instance and run the solver.  This will basically always be the same
# for all models that you run!  You can set tee=False to suppress the output.

opt = pe.SolverFactory('glpk')
instance = model.create_instance()
result = opt.solve(instance, tee=True)
instance.solutions.store_to(result)

    model; returning a clone of the current model instance.
GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/j6/vzm_9p_j4_x08rrgkr0f7cb84vgyj9/T/tmpi17yynib.glpk.raw
 --wglp /var/folders/j6/vzm_9p_j4_x08rrgkr0f7cb84vgyj9/T/tmpaqdkfa42.glpk.glp
 --cpxlp /var/folders/j6/vzm_9p_j4_x08rrgkr0f7cb84vgyj9/T/tmpd18g24gx.pyomo.lp
Reading problem data from '/var/folders/j6/vzm_9p_j4_x08rrgkr0f7cb84vgyj9/T/tmpd18g24gx.pyomo.lp'...
2 rows, 3 columns, 3 non-zeros
21 lines were read
Writing problem data to '/var/folders/j6/vzm_9p_j4_x08rrgkr0f7cb84vgyj9/T/tmpaqdkfa42.glpk.glp'...
15 lines were written
GLPK Simplex Optimizer, v4.65
2 rows, 3 columns, 3 non-zeros
Preprocessing...
1 row, 2 columns, 2 non-zeros
Scaling...
 A: min|aij| =  3.000e+00  max|aij| =  4.000e+00  ratio =  1.333e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 1
      0: obj =   0.000000000e+00 inf =   1.000e+00 (1)
      1: obj =  

In [125]:
# Get the objective function value from the optimal solution and print it.
obj_val = instance.obj.expr()
print('Optimal objective value = {}'.format(obj_val))

Optimal objective value = 0.666666666666666


In [126]:
# Get a list of optimal decision variable values.
x = []  # create an empty list to store values
for index in model.x_index.value_list:
    x.append(instance.x[index].value)
print('optimal x = {}'.format(x))

optimal x = [0.333333333333333, 0.0]
